## Ejemplo predicción de precios.

La base de datos se puede encontrar en:<p>
https://www.kaggle.com/harlfoxem/housesalesprediction/data<p>
Predicción del precio de casas en King County, USA.<p>
Veo lacorrelación entre la variable que quiero predecir (precio) y el resto:<p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics, linear_model

In [2]:
raw_data = pd.read_csv('data/kc_house_data.csv')
f, ax = plt.subplots(figsize=(4, 3))
corr = raw_data.corr()
corr.ix[1:, 1]

price            1.000000
bedrooms         0.308350
bathrooms        0.525138
sqft_living      0.702035
sqft_lot         0.089661
floors           0.256794
waterfront       0.266369
view             0.397293
condition        0.036362
grade            0.667434
sqft_above       0.605567
sqft_basement    0.323816
yr_built         0.054012
yr_renovated     0.126434
zipcode         -0.053203
lat              0.307003
long             0.021626
sqft_living15    0.585379
sqft_lot15       0.082447
Name: price, dtype: float64

Realizo un tratamiento de datos, transformo en variables categóricas (zipcode, grade, waterfront y condition) y posteriormente genero variables dummy. Elimino (id, date y long)

In [3]:
data = raw_data
data["zipcode"] = data["zipcode"].astype('category')
data["grade"] = data["grade"].astype('category')
data["waterfront"] = data["waterfront"].astype('category')
data["condition"] = data["condition"].astype('category')
data.drop(["id", "date", "long"], axis=1, inplace = True)
data = pd.get_dummies(data)

Divido la muestra en entrenamiento, validación y test.(60/20/20)

In [4]:
train, val, test = np.split(data.sample(frac=1, random_state=1),
                            [int(.6*len(data)), int(.8*len(data))])
y_train = train["price"]
y_val = val["price"]
y_test = test["price"]
X_train = train.ix[:, 1:103]
X_val = val.ix[:, 1:103]
X_test = test.ix[:, 1:103]

Calculo distintos modelos en función de el parámetro de regularización y la complejidad de la hipótesis <p>
En scikit-learn es $\alpha=\dfrac{1}{C}$ y el grado del polinomio lo tomo de la función PolynomialFeatures ($d$). <p>
Calculo la raiz del error cuadrático medio para los tres data sets.

In [5]:
list_C = list([0.1, 0.0001,0.00000001,  
               0.000000001, 0.000000005])
list_d = [1, 2]; count = 0
mse_train = np.zeros(len(list_C)*len(list_d))
mse_val = np.zeros(len(list_C)*len(list_d))
mse_test = np.zeros(len(list_C)*len(list_d))
degree = np.zeros(len(list_C)*len(list_d))
param_C = np.zeros(len(list_C)*len(list_d))
R_test = np.zeros(len(list_C)*len(list_d))
for d in list_d:
    MapF = preprocessing.PolynomialFeatures(degree=d)
    for C in list_C:
        reg = linear_model.Ridge(alpha = 1/C)
        reg.fit(MapF.fit_transform(X_train), y_train)
        mse_train[count] = np.sqrt(metrics.mean_squared_error(y_train,
                            reg.predict(MapF.fit_transform(X_train))))
        mse_val[count] = np.sqrt(metrics.mean_squared_error(y_val,
                          reg.predict(MapF.fit_transform(X_val))))
        mse_test[count] = np.sqrt(metrics.mean_squared_error(y_test, 
                           reg.predict(MapF.fit_transform(X_test))))
        R_test[count] = reg.score(MapF.fit_transform(X_test), y_test)
        degree[count] = d
        param_C[count]  = C
        count = count + 1

$\text{Observo alguno modelos y la raiz de su error cuadratico medio (RMSE).} \\
 \sqrt{J_\text{train}(\theta)} \text{ = Train RMSE} \\
 \sqrt{J_\text{val}(\theta)} \text{  = Validation RMSE} \\
 \sqrt{J_\text{test}(\theta)} \text{ = Test RMSE}$

In [6]:
matrix = np.matrix(np.c_[degree, param_C, np.round_(mse_train, 3), 
                         np.round_(mse_val, 3), np.round_(mse_test, 3), 
                         np.round_(R_test, 3)])
models = pd.DataFrame(data = matrix, columns = 
             ['d', 'parameter C', 'Train RMSE', 
              'Val RMSE', 'Test RMSE', 'R^2 Test'])
models.head(n=7)

,d,parameter C,Train RMSE,Val RMSE,Test RMSE,R^2 Test
0,1.0,1.000000e-01,154654.185,138077.410,155271.980,0.832
1,1.0,1.000000e-04,240817.986,214464.110,243528.825,0.586
2,1.0,1.000000e-08,259763.200,232956.935,262739.972,0.518
3,1.0,1.000000e-09,261464.051,233428.053,264295.415,0.512
4,1.0,5.000000e-09,260313.338,233390.852,263299.973,0.516
5,2.0,1.000000e-01,94770.163,151192.934,135241.450,0.872
6,2.0,1.000000e-04,105706.098,153093.041,136972.359,0.869


Selecciono el modelo con menor raiz del error cuadrático medio sobre el conjunto de validación. La capacidad real de generalización vendrá dada por el error en el conjunto de test dado que es totalmente independiente al modelo elegido. Si evaluasemos este error con el conjunto de validación tenderíamos a sobreestimarlo dado que ha sido un parámetro a la hora de elegir el modelo. <p>
Modelo con menor RMSE en el conjunto de validación

In [7]:
best_index = models['Val RMSE'].idxmin()
models.ix[best_index, :]

d              2.000000e+00
parameter C    1.000000e-08
Train RMSE     1.127296e+05
Val RMSE       1.325598e+05
Test RMSE      1.323642e+05
R^2 Test       8.780000e-01
Name: 7, dtype: float64

Mido la capacidad de generalización con $\sqrt{J_\text{test}(\theta)}$ y $R^2$ que es 0.878